# Calculations of the resistome datasets from nanopore, Sanger and PacBio
This notebook takes the Sanger and PacBio dataset and BLAST it against the CARD database. Next all the nanopore, Sanger and PacBio datasets are blast'ed against the Sanger data to get a sense of the sequence identity

* Requires [blast command line tool](https://blast.ncbi.nlm.nih.gov/Blast.cgi?PAGE_TYPE=BlastDocs&DOC_TYPE=Download)

In [3]:
%matplotlib inline
from collections import OrderedDict
import pandas as pd
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib import pyplot as plt
import os
from Bio import SeqIO
from types import *
import numpy as np
from subprocess import Popen, PIPE
import os
import logging

logger = logging.getLogger('CARDBlast')
logging.basicConfig()
logger.setLevel(logging.INFO)

CORES = 4

CARDdb = 'inputFiles/n.fasta.protein.homolog.fasta' #Card database

//anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:1011: UserWarning: Duplicate key in file "//anaconda/lib/python2.7/site-packages/matplotlib/mpl-data/stylelib/dark_background.mplstyle", line #9
  (fname, cnt))


## run poreFUME
The following block will execute poreFUME, be aware that this can take a long time depending on the amount of cores avialable (set with the ```--cores``` flag). The reads are split up by barcode over the cores. Also be sure to have run ```install.sh``` prior to running this notebook. In the ```poreFUME/``` directory you can run ```nosetests``` to check poreFUME is working correctly. Furthermore poreFUME relies on avialable paths, so if you did not add the paths to you ```~./profile``` run ```source env.sh``` to load them in the current shell.  
** The data in the ```output/``` directory will be overwritten by running poreFUME with the current ```--overwriteDemux --overwriteNanocorrect --overwriteCARD --overwriteNanopolish``` flags. If you just wish to analyze the data head over to the ```analyzeResistome.ipynb``` which uses the already generated data in ```output/``` **

In [26]:
import os
HERE = os.getcwd()
nanocorrectDir = os.path.join(HERE,'poreFUME','nanocorrect')

#run poreFUME (see integrationTest.sh for an short example run on 75 raw reads)
!python poreFUME/poreFUME.py inputData/testSet75.fasta inputData/pb_39.fasta --PacBioLegacyBarcode --cores 8 --pathCARD=inputData/n.fasta.protein.homolog.fasta --pathNanocorrect=---abspahgoeshere--/nanocorrect/ --pathRawreads=---abspahgoeshere--/test/data/testSet75 --overwriteNanocorrect --pathNanopolish=---abspahgoeshere--/nanopolish/ --overwriteNanopolish --overwriteDemux --overwriteCARD

!python poreFUME/poreFUME.py inputFiles/LejlaControl.2D.min500bp.fasta poreFUME/inputData/pb_39b.fasta  --barcodeEdge=60 --pathNanocorrect={nanocorrectDir} --PacBioLegacyBarcode --cores 8 --pathCARD=inputData/n.fasta.protein.homolog.fasta --pathNanocorrect=---abspahgoeshere--/nanocorrect/ --pathRawreads=---abspahgoeshereToENAFiles- --overwriteNanocorrect --pathNanopolish=---abspahgoeshere--/nanopolish/ --overwriteNanopolish --overwriteDemux --overwriteCARD
!python poreFUME/poreFUME.py inputFiles/poreCamp.2D.min500.fasta poreFUME/inputData/pb_39b.fasta  --pathNanocorrect={nanocorrectDir}  --barcodeEdge=120 --PacBioLegacyBarcode --cores 8 --pathCARD=inputData/n.fasta.protein.homolog.fasta --pathNanocorrect=---abspahgoeshere--/nanocorrect/ --pathRawreads=---abspahgoeshereToENAFiles- --overwriteNanocorrect --pathNanopolish=---abspahgoeshere--/nanopolish/ --overwriteNanopolish --overwriteDemux --overwriteCARD

In [4]:
def cmdExists(cmd):
    return any(
        os.access(os.path.join(path, cmd), os.X_OK)
        for path in os.environ["PATH"].split(os.pathsep)
    )
if not cmdExists('blastn'):
    raise  IOError('blastn not found in PATH, install from for example https://blast.ncbi.nlm.nih.gov/Blast.cgi?PAGE_TYPE=BlastDocs&DOC_TYPE=Download')
if not cmdExists('makeblastdb'):
    raise  IOError('makeblastdb not found in PATH, install from for example https://blast.ncbi.nlm.nih.gov/Blast.cgi?PAGE_TYPE=BlastDocs&DOC_TYPE=Download')

### load in the paths to all the files. 
In the python file ```paths.py``` a ```OrderedDict``` keeps track of the paths to the FASTA files. It is loaded in using the ```%load``` magic

In [ ]:
# %load paths.py
qPathOrdered = OrderedDict([
        
    ('S_raw','inputFiles/sanger.total.aftertrim.removeCT.min500bp.fasta'),
    ('S_ass','inputFiles/SangerAssembledAndNonAssembled.min500bp.withoutCandT.fasta'),


    ('PB_raw','inputFiles/PB.Cell1and2.raw.fasta'),
    ('PB_ass','inputFiles/PacBio.Correct.C1.fasta'),

    ('MI_raw','inputFiles/LejlaControl.2D.min500bp.fasta'),
    ('MI_demux'   , 'output/LejlaControl.2D.min500bp.afterBC.fasta'),
    ('MI_corr1'   , 'output/LejlaControl.2D.min500bp.afterNC1.fasta'),
    ('MI_corr2'   , 'output/LejlaControl.2D.min500bp.afterNC2.fasta'), 
    ('MI_polish'   , 'output/LejlaControl.2D.min500bp.afterNP.fasta'),
    ('MI_polishNoFilter'   , 'output/LejlaControl.2D.min500bp.afterNPnofilter.fasta'), #This file will be generated by calculateResistome.ipynb


    ('MIpc_raw', 'inputFiles/poreCamp.2D.min500.fasta'), 
    ('MIpc_demux' , 'output/poreCamp.2D.min500.afterBC.fasta'),  
    ('MIpc_corr1' , 'output/poreCamp.2D.min500.afterNC1.fasta'), 
    ('MIpc_corr2' , 'output/poreCamp.2D.min500.afterNC2.fasta'),
    #('MIpc_polish'   , 'output/poreCamp.2D.min500.afterNP.fasta'),
    #('MIpc_polishNoFilter'   , 'output/poreCamp.2D.min500.afterNPnofilter.fasta') #This file will be generated by calculateResistome.ipynb

        
      
    
    ])

annotationPathOrdered = OrderedDict([
        
    ('S_raw','output/annotation/sanger.total.aftertrim.removeCT.min500bp.fasta.annotated.csv'), 
    ('S_ass','output/annotation/SangerAssembledAndNonAssembled.min500bp.withoutCandT.fasta.annotated.csv'),


    ('PB_raw','output/annotation/PB.Cell1and2.raw.fasta.annotated.csv'), 
    ('PB_ass','output/annotation/PacBio.Correct.C1.fasta.annotated.csv'),

    ('MI_raw','output/annotation/LejlaControl.2D.min500bp/LejlaControl.2D.min500bp.annotated.csv'), 
    ('MI_demux'   , 'output/annotation/LejlaControl.2D.min500bp/LejlaControl.2D.min500bp.afterBC.annotated.csv'), 
    ('MI_corr1'   , 'output/annotation/LejlaControl.2D.min500bp/LejlaControl.2D.min500bp.afterNC1.annotated.csv'), 
    ('MI_corr2'   , 'output/annotation/LejlaControl.2D.min500bp/LejlaControl.2D.min500bp.afterNC2.annotated.csv'), 
    ('MI_polishNoFilter'   , 'output/annotation/LejlaControl.2D.min500bp.afterNPnofilter.fasta.annotated.csv'), 


    ('MIpc_raw', 'output/annotation/poreCamp.2D.min500/poreCamp.2D.min500.annotated.csv'), 
    ('MIpc_demux' , 'output/annotation/poreCamp.2D.min500/poreCamp.2D.min500.afterBC.annotated.csv'),  
    ('MIpc_corr1' , 'output/annotation/poreCamp.2D.min500/poreCamp.2D.min500.afterNC1.annotated.csv'),  
    ('MIpc_corr2' , 'output/annotation/poreCamp.2D.min500/poreCamp.2D.min500.afterNC2.annotated.csv'), 
    #('MIpc_polishNoFilter'   , 'output/annotation/poreCamp.2D.min500.afterNPnofilter.fasta.annotated.csv'), 
    

    
    ])


readLabels = OrderedDict([
        
    ('S_raw','Sanger raw'),
    ('S_ass','Sanger assembled'),


    ('PB_raw','PacBio raw'), 
    ('PB_ass','PacBio assembled'),


    ('MIpc_raw', 'nanopore lib. B:\n2D reads'), 
    ('MIpc_demux' , 'nanopore lib. B:\nafter debarcoding'),
    ('MIpc_corr1' , 'nanopore lib. B:\nafter nanocorrect round 1'), 
    ('MIpc_corr2' , 'nanopore lib. B:\nafter nanocorrect round 2'), 
    #('MIpc_polishNoFilter'   , 'nanopore lib. B:\nafter nanopolish')  ,

    ('MI_raw','nanopore lib. A: 2D reads'), 
    ('MI_demux'   , 'nanopore lib. A:\nafter debarcoding'),
    ('MI_corr1'   , 'nanopore lib. A:\nafter nanocorrect round 1'), 
    ('MI_corr2'   , 'nanopore lib. A:\nafter nanocorrect round 2')  ,
    ('MI_polishNoFilter'   , 'nanopore lib. A:\nafter nanopolish ')  

    
    ])

readLabelsSample = OrderedDict([
        
    ('S_raw','Sanger'), 
    ('S_ass','Sanger'),


    ('PB_raw','PacBio'), 
    ('PB_ass','PacBio'),


    ('MIpc_raw'   , 'nanopore lib. B'),
    ('MIpc_demux' , 'nanopore lib. B'), 
    ('MIpc_corr1' , 'nanopore lib. B'), 
    ('MIpc_corr2' , 'nanopore lib. B'), 
    #('MIpc_polishNoFilter'   , 'nanopore lib. B') ,
    
    ('MI_raw'     , 'nanopore lib. A'),
    ('MI_demux'   , 'nanopore lib. A'),
    ('MI_corr1'   , 'nanopore lib. A'), 
    ('MI_corr2'   , 'nanopore lib. A'),
    ('MI_polishNoFilter'   , 'nanopore lib. A') 

    
    ])

readLabelsWorkflow = OrderedDict([
        
    ('S_raw','raw'), 
    ('S_ass','assembled'),


    ('PB_raw','raw'), 
    ('PB_ass','assembled'),


    ('MIpc_raw', 'raw 2D'), 
    ('MIpc_demux' , 'after debarcoding'),
    ('MIpc_corr1' , 'after nanocorrect round 1'), 
    ('MIpc_corr2' , 'after nanocorrect round 2'),
    #('MIpc_polishNoFilter'   , 'after nanopolish')  ,

    ('MI_raw','raw 2D'), 
    ('MI_demux'   , 'after debarcoding'),
    ('MI_corr1'   , 'after nanocorrect round 1'), 
    ('MI_corr2'   , 'after nanocorrect round 2') ,
    ('MI_polishNoFilter'   , 'after nanopolish')  
    
    ])

pacbioMuxData = 'inputFiles/pacbioMux.p'


colorSet = {'MI': '#4daf4a',
        'PB': '#377eb8',
        'MIpc':'#ff7f00',
        'S':'#984ea3'}


## Combine nanocorrect and nanopolish data


In [23]:


def createFullPolish(polishedPath ,correctedPath,nofilterPath):
#def createFullPolish(polishedPath (=qPathOrdered['MI_polish']),correctedPath(=qPathOrdered['MI_corr2']),nofilterPath(=qPathOrdered['MI_polishNoFilter'])):
    """
    We ran nanopore in such as fashion that only high coverage reads are polished (>30).
    Reads that do not match this criteria (but can still have a high quality) are dropped from nanopolishing.
    Consequently we lose sequence diversity, therefore we combine all the non-polished reads (thus just taking the reads after two rounds of nanocorrect) with the polished-reads. 
    But in order to this we first need to 'calculate' which reads were not polished (for the next version we can just let poreFUME take care of this)
    """
    logger.info("Start merging the nanocorrect and nanopolish data")
    
    if not os.path.isfile(polishedPath):
        raise IOError('thisFilePath does not excist',polishedPath)

    if not os.path.isfile(correctedPath):
        raise IOError('thisFilePath does not excist',correctedPath)

    if os.path.isfile(nofilterPath):
        logger.error('The %s excist already, we are going to overwrite this!',nofilterPath)
        
    polishList = []
    for record in SeqIO.parse(polishedPath, "fasta"): #Read in the records of the nanopolished data
        polishList.append((record.id)) #Store all the name in a list

    #Quick sanity check if list is unique
    assert len(polishList) == len(set(polishList)), "The polishList record names are not unique! poreFUME gives unique names, is something wrong?"


    polishSet = set(polishList)

    nonPolished = []

    for record in SeqIO.parse(correctedPath, "fasta"): #Read in the records of the nanopolished data
        if record.id not in polishSet:
            #print record.id
            nonPolished.append(record) #Wrire record to a 'non-polished' list


    output_handle = open("nonpolished.fasta.tmp", "w")
    SeqIO.write(nonPolished, output_handle, "fasta")
    output_handle.close()

    #!cat {polishedPath} | grep ">" | wc -l
    !cat nonpolished.fasta.tmp {polishedPath} > {nofilterPath} 
    #!cat nonpolished.fasta.tmp | grep ">" | wc -l
    !rm nonpolished.fasta.tmp
    
    logger.info("Created a combined set of ")
    !cat {nofilterPath} | grep ">" | wc -l

#createFullPolish(qPathOrdered['MI_polish'],qPathOrdered['MI_corr2'],qPathOrdered['MI_polishNoFilter'])
createFullPolish(qPathOrdered['MIpc_polish'],qPathOrdered['MIpc_corr2'],qPathOrdered['MIpc_polishNoFilter'])


INFO:CARDBlast:Start merging the nanocorrect and nanopolish data
ERROR:CARDBlast:The output/poreCamp.2D.min500.afterNPnofilter.fasta excist already, we are going to overwrite this!


     582
    1810


INFO:CARDBlast:Created a combined set of 


    2392


# BLAST against the CARD database
The [CARD database](https://card.mcmaster.ca/) is a _"bioinformatic database of resistance genes, their products and associated phenotypes"_. The reads from the various sequencing platforms are annotated using BLAST against the nucleotide CARD database. Please cite [McArthur et al. 2013. The Comprehensive Antibiotic Resistance Database. Antimicrobial Agents and Chemotherapy, 57, 3348-3357.](http://www.ncbi.nlm.nih.gov/pubmed/23650175) when using. 

The ```calcSegments()``` function will prune the BLAST report so only the highest scoring (by bitscore) segments are reported back. This is more robust than ORF calling (data not shown).
_Note that this function takes quite a while on larger datasets (raw PacBio), even on multicore since calcSegements() is not optimized_

In [14]:
logger.info("Initialization done")

#########
#Functions
#########

def blastDatabase(queryFile,dbFile):
    """
    Returns a dataframe of a BLAST operation of the queryFile on the dbFile
    
    Parameters:
    queryFile = path to file that will be used as query in the BLAST process
    dbFile = path to file that will be used as database in the BLAST process
    """
    #Create a blast DB http://www.ncbi.nlm.nih.gov/books/NBK279688/
    
    
    assert type(queryFile) is StringType, "queryFile is not a string: %r" % queryFile 
    assert type(dbFile) is StringType, "databaseFile is not a string: %r" % dbFile 
        
    #Test inputs
    if not os.path.isfile(queryFile):
        raise IOError('queryFile does not excist',queryFile)
    
    if not os.path.isfile(dbFile):
        raise IOError('queryFile does not excist',dbFile)
    
    
    #Create dbase
    logger.info("Start builing blast database")
    process = Popen(['makeblastdb', '-in', str(dbFile), '-dbtype', 'nucl'], stdout=PIPE, stderr=PIPE)
    stdout, stderr = process.communicate()
    process.wait() #wait till finished
    logger.info(str(stdout))
    if stderr:
        logger.error(str(stderr))

    #Search with blast
    logger.info("Start BLASTing for subsample %s",queryFile)
    process = Popen(['blastn','-db',str(dbFile),'-query',str(queryFile),'-max_hsps', '1', '-max_target_seqs', '1000', '-num_threads', str(CORES), '-outfmt' ,'10','-out','blastn.tmp.output'], stdout=PIPE, stderr=PIPE)
    stdout, stderr = process.communicate()
    process.wait() #Wait till finished

    logger.info(str(stdout))
    if stderr:
        logger.error(str(stderr))
    logger.info("Finished BLASTing for subsample %s",queryFile)

    if not os.path.isfile('blastn.tmp.output'):
        raise RuntimeError('BLAST did not produce an alignment, one can implement an exception here. But for now stop')

    #Read in blast results
    dfBlast = None
    dfBlast = pd.read_csv('blastn.tmp.output',names=['qseqid' ,'sseqid' ,'pident', 'length', 'mismatch', 'gapopen', 'qstart', 'qend' ,'sstart' ,'send','evalue', 'bitscore']) #load demuxed and corrected Minion dbase blasted by Sanger



    os.remove(str(dbFile) +'.nhr') #cleanup subsample file blastDB
    os.remove(str(dbFile) +'.nin') #cleanup subsample file blastDB
    os.remove(str(dbFile) +'.nsq') #cleanup subsample file blastDB

    os.remove('blastn.tmp.output') #cleanup blast output
    
    return dfBlast



def calcGeneLength(row):
    """
    The CARD database contains the position of the subject gene in the header name, this is originally stored in this script in 
    id2 but split into a subjectGeneSTart and subjectGeneEnd. Based on this we can calculate the original gene length

    """
   
    return abs(int(row['subjectGeneStart'])- int(row['subjectGeneEnd']))




def calcCoverage(row):
    """
    We can calculate the coverage of the alignment by dividing the length of the alignemnt over the subjectGeneLength
    There is a glitch in the length of the header and the real length of the DNA in the card database, so limit of at 100%
    """
    #return (int(row['length'])/ float(row['subjectGeneLength']))*100

    return (int(row['length'])/ float(row['subjectGeneLength']))*100 if (int(row['length'])/ float(row['subjectGeneLength']))*100 < 100 else 100


def calcSegments(thisDF):
    """
    Calculates the most relevant hit for each segment on the read
    
    Paramters:
    thisDF = dataframe with the BLAST result for an INDIVIDUAL query. So don't pass the full BLAST table, but only from 1 qseqid, ie. thisDF[thisDF.qseqid == thisSeqid], where thisSeqid is the current seqid of interest
    """
    #from PIL import Image, ImageDraw #For viz purposes
    
    #im = Image.new('RGBA', (6000, 1000), (0, 0, 0, 0))  #initialize a debug drawing screen
    #draw = ImageDraw.Draw(im) 
    thisDF.reset_index(drop=True,inplace=True)

    dy = 50 #off set for debug drawing
    highscore = [] #Store the coordinates of the visited positions in a [start,end] format
    highindex = [] #Store the index number (=blast hit) of each visited position
    #for row in thisDF.itertuples(): #go through all the rows of the dataframe, not the most effient way! When this scales up write a new implemetation based on sorting

   
    for index,qstart,qend in zip(thisDF.index.values, thisDF.qstart, thisDF.qend): #this speeds up 5x compared to using iterrows(). However it is still slow. 
        #print row
        if len(highscore) == 0: #First hit
            highscore.append([qstart,qend]) #Add position to the position list
            highindex.append(index)  #also keep track of the index
            #draw.line((row['qstart'],dy,row['qend'],dy), fill=(255,255,255)) #draw bright
        else:
            doesOverlap = False
            for thisScore in highscore: #Go through all the set positions
                if calcOverlap(thisScore[0],thisScore[1],qstart,qend) > 0: #If there is an overlap 
                    #print 'Overlapping, skip'
                    doesOverlap = True
                    break #Exit loop, no need to continue
            if doesOverlap == False: #We found a new segment
                highscore.append([qstart,qend]) #Add position to list
                #draw.line((row['qstart'],dy,row['qend'],dy), fill=(255,255,255))  #Draw bright
                highindex.append(index)

            else: #Old segment
                pass
                #draw.line((row['qstart'],dy,row['qend'],dy), fill=(55,55,55))     #Draw darker for debugging



        #print row['qstart'],row['qend']


        dy = dy + 2
    #im.save('out.png',"PNG")
    #im.show() #show a debug figure
 
    return thisDF.iloc[highindex] #Return a dataframe with the most relevant hit on each segment

def calcOverlap(a,b,c,d):
    """
    Returns the overlap between two segments (ab) vs (cd)
    
    Parameters:
    a = start position of segment AB
    b = end position of segment AB
    c = start position of segment CD
    d = end position of segment CD
    
    Can be in any direction and any order
    """
    return min([max([a,b]), max([c,d])]) - max([min([c,d]), min([a,b])]) #calculate overlap between two segments



###########
## Production loop
###########

#for thisQueryFile in qpath:
for thisRunName, thisQueryFile in qPathOrdered.items():    
    
    if (thisRunName.split('_')[0] == 'S') | (thisRunName.split('_')[0] == 'PB'): #MI and #MIpc are already done by poreFUME (except the nofilter)
        pass
    else:
        if (thisRunName.split('_')[1] == 'polishNoFilter'):  #the MI_polishNoFilter and MIpc_polishNoFilter we just generated, so they need to be annotated
            pass
        else:
            continue

    logger.info('Starting CARD analysis loop with %s stored at %s',thisRunName,thisQueryFile)
    dfBlast =  blastDatabase(thisQueryFile,CARDdb) 

    #Parse the CARD database identifiers
    logger.info("Parse BLAST data")
    dfBlast[['gb','id1','id2','ARO','GeneName']]= dfBlast.sseqid.str.split('|',expand=True) #Split the CARD header out
    dfBlast[['subjectGeneStart','subjectGeneEnd']]= dfBlast.id2.str.split('-',expand=True) #Split the CARD gene length
    dfBlast = dfBlast.drop('id2',1) #remove id2 redundand
    dfBlast = dfBlast.drop('gb',1) #remove gb, does not contain information

    dfBlast['subjectGeneLength'] = dfBlast.apply(calcGeneLength, axis=1)  

    dfBlast['coverageOfSubjectGene'] = dfBlast.apply(calcCoverage, axis=1)


    dfBlastSort = dfBlast.sort_values('bitscore',ascending=False) #sort by bitscore (in case it is not)

    dfBlastSort = dfBlastSort.reset_index(drop=True) #Reindex based on bitscore
    logger.info("Identified BLAST hits: " + str(dfBlastSort.shape[0]))

    logger.info("Start identifing relevant segments")

    logger.info("Finished identifing relevant segments")

    #In order to calculate each the most relevant BLAST hits we go through each query sequence and calculate the relevant hits
    dfTotal = pd.DataFrame() #Initialze the final table

    for thisSeqid in dfBlastSort.qseqid.unique(): #go through each query sequence individually
        
        dfTotal = dfTotal.append(calcSegments(dfBlastSort[dfBlastSort.qseqid == thisSeqid]), ignore_index=True) #Make a subset of the BLAST table containing only the current query seqience, next calculate the relevant blast hits and add them to the dfTotal table

    logger.info("Identified single hits: " + str(dfTotal.shape[0]))

    ###Save results in querySuffix + '.annotated.csv'


    dfTotal =  dfTotal.add_prefix('CARD:') 

    queryPostfix, querySuffix = os.path.split(thisQueryFile) #Parse filename

    #Make sure there a directory to store the result
    process = Popen(['mkdir', 'output/annotation'], stdout=PIPE, stderr=PIPE)
    stdout, stderr = process.communicate()
    process.wait() #wait till finished
    logger.info(str(stdout))
    if stderr:
        logger.error(str(stderr))


    dfTotal.to_csv('output/annotation/' + querySuffix + '.annotated.csv')

    logger.info("Saved annotationed in: output/annotation/" + querySuffix + ".annotated.csv")

INFO:CARDBlast:Initialization done
INFO:CARDBlast:Starting CARD analysis loop with MI_polishNoFilter stored at output/LejlaControl.2D.min500bp.afterNPnofilter.fasta
INFO:CARDBlast:Start builing blast database
INFO:CARDBlast:

Building a new DB, current time: 11/19/2016 13:40:03
New DB name:   /Users/evand/Downloads/nanorevision/production_setup/poreFUME_paper/inputFiles/n.fasta.protein.homolog.fasta
New DB title:  inputFiles/n.fasta.protein.homolog.fasta
Sequence type: Nucleotide
Keep Linkouts: T
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 2181 sequences in 0.149022 seconds.

INFO:CARDBlast:Start BLASTing for subsample output/LejlaControl.2D.min500bp.afterNPnofilter.fasta
INFO:CARDBlast:
INFO:CARDBlast:Finished BLASTing for subsample output/LejlaControl.2D.min500bp.afterNPnofilter.fasta
INFO:CARDBlast:Parse BLAST data
INFO:CARDBlast:Identified BLAST hits: 307663
INFO:CARDBlast:Start identifing relevant segments
INFO:CARDBlast:Finished identifing rele

# BLAST against Sanger dataset
In order to investigate the sequence identity of the obtained reads a Sanger dataset is avialble whith contains a subset of the total colonies. 

In [16]:
def blastDatabaseSanger(queryFile,dbFile,fillUp = False,maxHits = 1):
    """
    Returns a dataframe of a BLAST operation of the queryFile on the dbFile. Optionally the non-matched queries can be added with pident, length and bitscore = 0
    
    Parameters:
    queryFile = path to file that will be used as query in the BLAST process
    dbFile = path to file that will be used as database in the BLAST process
    fillUp = False, turn on to add non-matched query sequences
    maxHits = 1, the max number of hits per query.
    """
    #Create a blast DB http://www.ncbi.nlm.nih.gov/books/NBK279688/
    
    
    assert type(queryFile) is StringType, "queryFile is not a string: %r" % queryFile 
    assert type(dbFile) is StringType, "databaseFile is not a string: %r" % dbFile 
        
    #Test inputs
    if not os.path.isfile(queryFile):
        raise IOError('queryFile does not exist',queryFile)
    
    if not os.path.isfile(dbFile):
        raise IOError('queryFile does not exist',dbFile)
    
    
    #Create dbase
    logger.info("Start builing blast database")
    process = Popen(['makeblastdb', '-in', str(dbFile), '-dbtype', 'nucl'], stdout=PIPE, stderr=PIPE)
    stdout, stderr = process.communicate()
    process.wait() #wait till finished
    logger.info(str(stdout))
    if stderr:
        logger.error(str(stderr))

    #Search with blast
    logger.info("Start BLASTing for subsample %s",queryFile)
    process = Popen(['blastn','-db',str(dbFile),'-query',str(queryFile),'-max_hsps', '1', '-max_target_seqs', str(maxHits), '-num_threads', str(CORES), '-outfmt' ,'10','-out','blastn.tmp.output'], stdout=PIPE, stderr=PIPE)
    stdout, stderr = process.communicate()
    process.wait() #Wait till finished

    logger.info(str(stdout))
    if stderr:
        logger.error(str(stderr))
    logger.info("Finished BLASTing for subsample %s",queryFile)

    if not os.path.isfile('blastn.tmp.output'):
        raise RuntimeError('BLAST did not produce an alignment, one can implement an exception here. But for now stop')

    #Read in blast results
    dfBlast = None
    dfBlast = pd.read_csv('blastn.tmp.output',names=['qseqid' ,'sseqid' ,'pident', 'length', 'mismatch', 'gapopen', 'qstart', 'qend' ,'sstart' ,'send','evalue', 'bitscore']) #load demuxed and corrected Minion dbase blasted by Sanger

    if fillUp:
        queryList = []
        queryLength = []
        for record in SeqIO.parse(queryFile, "fasta"):
            queryList.append(record.id)
            queryLength.append( len(record.seq))

        blastList = dfBlast.qseqid.tolist()


        nohitList = set(queryList) - set(blastList)
        for thisHit in nohitList:

            thisAddition = {
            'qseqid': thisHit,
            'length' : 0,
            'pident': 0,
            'bitscore': 0
            }

            dfBlast = dfBlast.append(thisAddition,ignore_index=True)

        #Sanity check
        if len(queryList) == dfBlast.shape[0]:
            logger.info('OK queryList and blast result are same length. Continue')
        else:
            raise ValueError('the resulting dataframe and the initial query list are not the same length!')
        


    os.remove(str(dbFile) +'.nhr') #cleanup subsample file blastDB
    os.remove(str(dbFile) +'.nin') #cleanup subsample file blastDB
    os.remove(str(dbFile) +'.nsq') #cleanup subsample file blastDB

    os.remove('blastn.tmp.output') #cleanup blast output
    
    return dfBlast

In [17]:
for thisRunName, thisQueryFile in qPathOrdered.items():
    if thisRunName == 'S_raw':
        continue
        
    thisDF = blastDatabaseSanger(thisQueryFile,qPathOrdered['S_raw'],fillUp=False)
    process = Popen(['mkdir', 'output/blastAgainstSanger'], stdout=PIPE, stderr=PIPE)
    stdout, stderr = process.communicate()
    process.wait() #wait till finished
    logger.info(str(stdout))
    if stderr:
        logger.error(str(stderr))

    queryPostfix, querySuffix = os.path.split(thisQueryFile) #Parse filename

    thisDF.to_csv('output/blastAgainstSanger/' + querySuffix + '.againstSanger.csv')

    
    logger.info("Saved annotationed in: output/blastAgainstSanger/" + querySuffix + ".againstSanger.csv")

INFO:CARDBlast:Start builing blast database
INFO:CARDBlast:

Building a new DB, current time: 11/19/2016 13:47:48
New DB name:   /Users/evand/Downloads/nanorevision/production_setup/poreFUME_paper/inputFiles/sanger.total.aftertrim.removeCT.min500bp.fasta
New DB title:  inputFiles/sanger.total.aftertrim.removeCT.min500bp.fasta
Sequence type: Nucleotide
Keep Linkouts: T
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 1568 sequences in 0.102869 seconds.

INFO:CARDBlast:Start BLASTing for subsample output/LejlaControl.2D.min500bp.afterNPnofilter.fasta
INFO:CARDBlast:
INFO:CARDBlast:Finished BLASTing for subsample output/LejlaControl.2D.min500bp.afterNPnofilter.fasta
INFO:CARDBlast:
ERROR:CARDBlast:mkdir: output/blastAgainstSanger: File exists

INFO:CARDBlast:Saved annotationed in: output/blastAgainstSanger/LejlaControl.2D.min500bp.afterNPnofilter.fasta.againstSanger.csv
INFO:CARDBlast:Start builing blast database
INFO:CARDBlast:

Building a new DB, current t